In [1]:
"""Key: p = pattern=input_list; node = smallest tree; chldrn = child edges; 
tlink = test link edge_type(branch's name)"""

"""In our learning algorithm, we will need to 
manipulate patterns: – test two patterns to see if they match or are equal.""" 

"""p_equal tests patterns equality. Two patterns 
are equal if they contain exactly the same items (including the end marker)"""

def p_equal(p1, p2):
    return p1 == p2

In [2]:
""" p_match test if the 2 pattern match. Examples of matching: 
– <a b c> matches <a b c d> 
– <a $> does not match <a b c>
– <a b c> does not match <a c b> """

def p_match(p1, p2):
    if len(p1) > 0 and p1[-1] == '$':
        if len(p1) - 1 == len(p2):
            return p1[:-1] == p2
    else:
        if len(p1) <= len(p2):
            return p1 == p2[:len(p1)]
    return False

In [3]:
"""match tests"""
print( p_match(['a','b','c'],['a','b','c','d']) )
print( p_match(['a','$'],['a','b','c']) )
print( p_match(['a','b','c'],['a','c','b']) )

True
False
False


In [4]:
"""The difference between two non-matching patterns is the 
part of the SECOND pattern after the common part of the pattern is removed
– <a b c> taken from <a b d c> = < d c > 
– <a b d c> taken from <a b c> = < c > 
– <a b c d> taken from <a c b d $> = < c b d $>"""

def p_diff(p1, p2):                 #below we look at example [a b d c] taken from [a b c] => [c] 
    for i in range(len(p2)):        #increment up to 3 non-inclusive (length of abc))
        if i >= len(p1) or p1[i] != p2[i]:       #if index of letter greater than pattern1 length
                                             #OR one of the letters on both patterns do not match
            break                            #stop the above loop
    return p2[i:]                #return remainder of p2, starting with the 1st non-matching letter

In [5]:
"""difference tests"""
print( p_diff(['a','b','c'],['a','b','d','c']) )
print( p_diff(['a','b','d','c'],['a','b','c']) )
print( p_diff(['a','b','c','d'],['a','c','b','d','$']) )
print( p_diff([],['a','b','c']) )
print( p_diff(['a', 'b', 'c'], ['a','b', '$']) )
#print( p_diff(['a','b','c'],[])) #Error: local variable 'i' referenced before assignment

['d', 'c']
['c']
['c', 'b', 'd', '$']
['a', 'b', 'c']
['$']


            Retrieval Process flow chart. Test links = chldrn.
![](files/Retrieval_process.jpg)

In [6]:
""" Retrieval Process function. Takes pattern, returns node.
UPPER "NO->Return" - never enters the while loop below; LOWER "NO->Return" implies following
the while loop."""

def retrieve(p):
    node = tree[0] #start at the 1st or root node (the latter skips the loop below in the beginning)
    i = 0                          #look for chldrn/test-links 
    while i < len(node['chldrn']): #while iterator < length of chldrn list
        l = node['chldrn'][i]
        if p_match(tree[l]['path'], p): #check for match between pattern and chldrn and if TRUE
            node = tree[l]              #then foolow the flow chart> current node = descendant node
            i = 0
        else:
            i += 1                       #if no match go on to the next node
    return node

In [7]:
"""Retrieval tests"""
# tree = [{'tlink': ['root'], 'chldrn': [1, 2], 'path': []}, #работaЁт!
#  {'tlink': ['A'], 'image': ['A', 'B'], 'chldrn': [], 'path': ['A']},
#  {'tlink': ['B'], 'image': [], 'chldrn': [], 'path': ['B']}]
tree = [
    {'tlink': ['root'], 'chldrn': [1, 2, 3], 'path': []},
    {'tlink': ['A'], 'image': ['A', 'B', 'C'], 'chldrn': [4], 'path': ['A']},
    {'tlink': ['B'], 'image': [], 'chldrn': [], 'path': ['B']},
    {'tlink': ['C'], 'image': [], 'chldrn': [], 'path': ['C']},
    {'tlink': ['B'], 'image': ['A','B'], 'chldrn': [], 'path': ['A', 'B']}
]
# tree = [{'tlink': ['root'], 'chldrn': [1], 'path': []},
#  {'tlink': 'A', 'image': [], 'chldrn': [], 'path': ['A']}]

print(retrieve(['A','B','C','$']) ) 
print("Recalled", retrieve(['A','B','C','$'])['image'], "for ABC$") # "Recognise" in CHREST GUI; NB need to add code for the root case 
print(retrieve(['D']) )



{'tlink': ['B'], 'image': ['A', 'B'], 'chldrn': [], 'path': ['A', 'B']}
Recalled ['A', 'B'] for ABC$
{'tlink': ['root'], 'chldrn': [1, 2, 3], 'path': []}


In [8]:
"""Learning Process
(1) The pattern is sorted to a node (what does this even mean?:))
(2) The pattern is compared with the image of the node reached 
(3) If the image matches the pattern, then familiarisation occurs 
(4) If the image mismatches the pattern, or the node is the rootnode, then discrimination occurs"""

def learn(p):
    node = retrieve(p)
    if node['tlink'] != ['root'] and p_match(node['image'], p): #if non-root image DOES match input pattern
        #print("Learn -> Familiarisation!")
        familiarisation(node, p)                         #never happens for root node!
    else:                                               #if non-root image does NOT match input pattern
        #print("Learn -> Descrimination!")
        discrimination(node, p)

In [9]:
""" learn tests """
tree = [
    {'tlink': ['root'], 'chldrn': [1], 'path': []}, #Fail? но итоговый выхлоп верный
    {'tlink': ['A'], 'image': ['A'], 'chldrn': [], 'path': ['A']} # 'A' уже в image, нужен новый node для 'B' = discrimanation
]
learn(['A', 'B', '$'])

NameError: name 'familiarisation' is not defined

In [10]:
"""Discrimination Process
Discrimination is how new nodes get added to the network. We take the part of the input pattern 
not used so far in sorting (the difference pattern), and attempt to sort it through the network. 
1. If we stay at the root, then learn a new primitive 
2. Get the image of the retrieved node: 
    a). If it is empty, familiarise with the difference pattern 
    b). Else, make a new test link: the test is the retrieved image (without any end marker)
    the image of the new node is the set of tests needed to reach it.
"""
'''
def discrimination(node, p):
    diff = p_diff(node['path'], p)
    if node['tlink'] == ['root']: #if at the root:
        tree.append({           #attach a subtree with the following params
            'tlink': [p[0]],      #tlink/test-link name = primitive = FIRST letter of the sorted pattern
            'image': [],
            'chldrn': [],
            'path': [p[0]]      #path list to include the same FIRST letter
        })
         #add index of the added primitive tlink (add 2-1=1 if root, since root tree was appended with another tree in lines above)
        tree[0]['chldrn'].append(len(tree) - 1)
    else:
        
        if len(node['image']) == 0:          
            familiarisation(node, diff)      #if image empty, familiarise with the difference pattern
        else:
            retr_chunk = retrieve(diff)
            if retr_chunk['tlink'] == ['root']:
                
            tlink = diff
            if tlink[-1] == '$':
                tlink.pop()
            if len(tlink) == 0:
                df = p_diff(node['path'], p)
                if df == ['$'] or len(df) == 0:
                    return
                tlink = [df[0]]
            
            #print("discr tlink=", tlink)
            #else: make new test link; 
            #the test is the retrieved image (without any end marker), 
            #the image of the new node is the set of tests needed to reach it
            tree.append({
                'tlink': tlink,
                'image': node['path'] + tlink,
                'chldrn': [],
                'path': node['path'] + tlink,
            })
            node['chldrn'].append(len(tree) - 1)
            #print('node:', node)
            #print("node[chldrn]:", node['chldrn'])
            #print('tree length:', len(tree))
'''
pass

In [11]:
def discrimination(node, p):
    new_information = p_diff(node['path'], p)
    #print('new inf', new_information)
    if len(new_information) == 0:
        if retrieve(new_information)['tlink'] == new_information:
            pass # add test #Strange empty $s added to visual in Peter's ver
        else:
            tree.append({           #attach a subtree with the following params
                'tlink': new_information,      #tlink/test-link name = primitive = FIRST letter of the sorted pattern
                'image': new_information,
                'chldrn': [],
                'path': [p[0]]      #path list to include the same FIRST letter
            })
            node['chldrn'].append(len(tree) - 1)
    else:
        retr_chunk = retrieve(new_information)
        #print('retr chunk', retr_chunk)
        if retr_chunk['tlink'] == ['root']:
            tree.append({           #attach a subtree with the following params
                'tlink': [new_information[0]],      #tlink/test-link name = primitive = FIRST letter of the sorted pattern
                'image': [],
                'chldrn': [],
                'path': [new_information[0]]      #path list to include the same FIRST letter
            })
             #add index of the added primitive tlink (add 2-1=1 if root, since root tree was appended with another tree in lines above)
            tree[0]['chldrn'].append(len(tree) - 1)
        elif p_match(retr_chunk['path'], new_information):
            path = node['path'] + retr_chunk['path']
            tree.append({
                'tlink': retr_chunk['path'],
                'image': path,
                'chldrn': [],
                'path': path.copy()
            })
            node['chldrn'].append(len(tree) - 1)
        else:
            path = node['path'] + retr_chunk['tlink']
            tree.append({
                'tlink': retr_chunk['tlink'],
                'image': path,
                'chldrn': [],
                'path': path
            })
            node['chldrn'].append(len(tree) - 1)

In [12]:
['d'] +['a']

['d', 'a']

In [13]:
""" discrimination tests"""
tree = [
    {'tlink': ['root'], 'chldrn': [1], 'path': []},
    {'tlink': ['A'], 'image': ['A'], 'chldrn': [], 'path': ['A']}
]
discrimination(tree[0], ['B'])
for line in tree:
    print(line)
tree = [
    {'tlink': ['root'], 'chldrn': [1, 2, 3], 'path': []},
    {'tlink': ['A'], 'image': ['A', 'B', 'C'], 'chldrn': [], 'path': ['A']},
    {'tlink': ['B'], 'image': [], 'chldrn': [], 'path': ['B']},
    {'tlink': ['C'], 'image': [], 'chldrn': [], 'path': ['C']},
]
discrimination(tree[1], ['A', 'B', '$'])
for line in tree:
    print(line)

{'tlink': ['root'], 'chldrn': [1, 2], 'path': []}
{'tlink': ['A'], 'image': ['A'], 'chldrn': [], 'path': ['A']}
{'tlink': ['B'], 'image': [], 'chldrn': [], 'path': ['B']}
{'tlink': ['root'], 'chldrn': [1, 2, 3], 'path': []}
{'tlink': ['A'], 'image': ['A', 'B', 'C'], 'chldrn': [4], 'path': ['A']}
{'tlink': ['B'], 'image': [], 'chldrn': [], 'path': ['B']}
{'tlink': ['C'], 'image': [], 'chldrn': [], 'path': ['C']}
{'tlink': ['B'], 'image': ['A', 'B'], 'chldrn': [], 'path': ['A', 'B']}


In [14]:
""" Familiarisation is how new information is added to the chunks in the network.
We take the difference between the image of the retrieved node and the input 
pattern and sort it through the network.
Three things may happen:
1. We get the root node back, then we have to learn a new primitive, using discrimination 
2. If retrieved image is empty or larger than difference, we just add the first item 
from difference to (first) image.
3. Otherwise, add the first item from retrieved image to the first image.
"""
def familiarisation(node, p):
    diff = p_diff(node['image'], p)
    if len(diff) == 0 or diff == ['$']: #do nothing if no diff between image and input pattern
        return
    if diff[-1] == '$':
        diff.pop()
    first = retrieve(diff)               #retrieve node using diff
    if first['tlink'] == ['root']:
        discrimination(first, diff)     
    elif len(first['image']) == 0 or len(first['image']) > len(diff):
        node['image'].append(diff[0])
    else:
        first['image'].append(diff[0])

In [15]:
def print_tree(node, idx=0, level=0):
    if idx == 0:
        print('root')
    else:
        indent = '-------' * level
        tlink = '< ' + ' '.join(node['tlink']) + ' >'
        image = '< ' + ' '.join(node['image']) + ' >'
        text = 'Node: ' + str(idx)
        print(indent + tlink, text, image)
    for cidx in node['chldrn']:
        print_tree(tree[cidx], cidx, level + 1)

## Examples of CHREST in action
Run one of the cells below (each one contains a series of pattern inputs), followed by the very last code cell at the very bottom of the page. 

In [18]:
inp = tuple(map(list, (
    'AB$', 'AB$', 'AB$', 'AB$', 'AB$', 'AB$',
    'ABC$', 'ABC$', 'ABC$',
    'AB$', 'AB$','AB$', 'AB$',
    'DZP$', 'DZP$', 'DZP$', 'DZP$', 'DZP$', 'DZP$', 'DZP$', 'DZP$',         #ctrl+/ to comment-uncomment 
    'ABCD$', 'ABCD$',
    'ABC$', 'ABC$', 'ABC$',
    'ABCD$', 'ABCD$',
    'ABC$',
    'ABCD$',
    'ABCDE$', 'ABCDE$',
    'ABCD$'
)))

tree = [                           #run from here to reset tree
  {
    'tlink': ['root'],
    'chldrn': [],
    'path': []
  }
]

In [16]:
inp = tuple(map(list, (
    'ABC$', 'ABC$', 'ABC$', 'ABC$', 'ABC$', 'ABC$',
    'AB$',
    'DEF$', 'DEF$', 'DEF$', 'DEF$',
    'DAB$'
)))

tree = [                           #run from here to reset tree
  {
    'tlink': ['root'],
    'chldrn': [],
    'path': []
  }
]

In [20]:
inp = tuple(map(list, (
    'ABC$', 'ABC$', 'ABC$', 'ABC$', 'ABC$', 'ABC$',
    'DEF$', 'DEF$', 'DEF$', 'DEF$', 'DEF$', 'DEF$',
    'DAC$'
)))

tree = [                           #run from here to reset tree
  {
    'tlink': ['root'],
    'chldrn': [],
    'path': []
  }
]

In [22]:
inp = tuple(map(list, (
    'AB$', 'AB$', 'AB$', 'AB$',
    'A$',
    'AC$','AC$',
    'A$'

)))

tree = [                           #run from here to reset tree
  {
    'tlink': ['root'],
    'chldrn': [],
    'path': []
  }
]

In [24]:
inp = tuple(map(list, (
    'AB$', 'AB$', 'AB$', 'AB$','AB$',
    'AC$', 'AC$', 'AC$', 'AC$', 
    'BB$', 'BB$',
    'BAC$'
)))

tree = [                           #run from here to reset tree
  {
    'tlink': ['root'],
    'chldrn': [],
    'path': []
  }
]

In [19]:
for p in inp:
    learn(p)
    #print('learn', p)
    #print_tree(tree[0]) #UNCOMMENT this command to see the full sequence of learning steps
print_tree(tree[0])

root
-------< A > Node: 1 < A B C >
--------------< B > Node: 4 < A B C D >
---------------------< C > Node: 8 < A B C D E >
----------------------------< D > Node: 11 < A B C D >
-------< B > Node: 2 <  >
-------< C > Node: 3 <  >
-------< D > Node: 5 < D Z P >
-------< Z > Node: 6 <  >
-------< P > Node: 7 <  >
-------< $ > Node: 9 <  >
-------< E > Node: 10 <  >
